In [3]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("/home/a_parida/MantaFlow/manta/tensorflow/tools/")
import uniio
import glob
basePath = 'data/'

In [4]:
trainingEpochs = 10000
batchSize      = 10
inSize         = 64 * 64 * 1

In [59]:
vel_files= glob.glob('./data/**/*.uni', recursive=True)
# load data
velocities = []

In [60]:
for uniPath in vel_files:
    header, content = uniio.readUni(uniPath)# returns [Z,Y,X,C] np array
    h = header['dimX']
    w  = header['dimY']
    arr = content[:, ::-1, :, :]  # reverse order of Y axis
    arr = np.reshape(arr, [w, h, 3])
    arr=arr[:,:,1:]# discard Z from [Z,Y,X]
    velocities.append( arr )
    
loadNum = len(densities)
if loadNum<200:
	print("Error - use at least two full sims, generate data by running 'manta ./manta_genSimSimple.py' a few times..."); exit(1)

velocities = np.reshape( velocities, (len(velocities), 64,64,2) )

print("Read uni files, total data " + format(velocities.shape) )
valiSize = max(100, int(loadNum * 0.1)) # at least 1 full sim...
valiData = velocities[loadNum-valiSize:loadNum,:]
velocities  = velocities[0:loadNum-valiSize,:]
print("Split into %d training and %d validation samples" % (velocities.shape[0], valiData.shape[0]) )
loadNum = velocities.shape[0]

Read uni files, total data (600, 64, 64, 2)
Split into 501 training and 99 validation samples


In [65]:
def convolution2d(input, biases, weights, strides, padding_kind='SAME'):
    input = tf.nn.conv2d(input, weights, strides, padding=padding_kind)
    input = tf.nn.bias_add(input, biases)
    input = tf.nn.leaky_relu(input)
    return input

In [61]:
# the network structure 
xIn = tf.placeholder(tf.float32, shape=[None, 64,64, 2])
y = tf.placeholder(tf.float32, shape=[None, 64*64*2])

In [ ]:
weights1=tf.Variable(tf.random_normal([5,64*64*2 ], stddev=0.01))

In [75]:
xIn = tf.reshape(x, shape=[-1, 64*64*2 ]) # flatten
weights1=tf.Variable(tf.random_normal([5,64*64*2 ], stddev=0.01))
bias1=tf.Variable(tf.random_normal([5], stddev=0.01))
stride1=4                    
out1=convolution2d(xIn,bias1,weights1,stride1)

TypeError: Expected list for 'strides' argument to 'conv2d' Op, not 4.

In [ ]:
weights2=tf.Variable(tf.random_normal([6*6*4], stddev=0.01))
bias2=tf.Variable(tf.random_normal([4*4*4], stddev=0.01))
stride2=[1,4,1,1]                    
out2=convolution2d(out1,bias2,weights2,stride2)